In [ ]:
# 네이버 기사 데이터 수집
# API : application programming interface
# naver(papago, keyword api), kakao(oauth, profile, kakotalk msg)
# selenium

# 인코딩 h -> c
# a > 0000 (encoding), 0000 > a (decoing)
# ascII : 영문알파벳, 숫자, 특수문자 (4Byte)
# euc-kr : 한국어도 (8byte)
# UTF-8 : 모든언어 (16Byte)

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# 1. 웹페이지 분석

In [6]:
category, date, page = 100, 20211105, 1
url = 'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)
url

'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1=100&date=20211105&page=1'

In [ ]:
# 2. 기사의 제목, 링크 데이터 수집

In [8]:
headers = {'user-agent' :'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [9]:
dom = BeautifulSoup(response.text, "html.parser")

In [16]:
selector = "#main_content > div.list_body.newsflash_body > ul > li"
elements = dom.select(selector)
len(elements)

20

In [18]:
element = elements[0]

In [25]:
data = {
    'title' : element.select('a')[1].text.strip(),
    'link' : element.select('a')[1].get('href'),
}
data

{'title': '‘MB·박근혜 사면’ 반대 48%, 찬성 44%',
 'link': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=005&aid=0001482244'}

In [ ]:
# 3. 기사내용 데이터 수집

In [34]:
datas = []
for element in elements:
    datas.append({
    'title' : element.select('a')[-1].text.strip(),
    'link' : element.select('a')[-1].get('href'),
    })
df = pd.DataFrame(datas)
df.tail(2)

,title,link
18,특별법안 제출하는 국민의힘,https://news.naver.com/main/read.naver?mode=LS...
19,"국민의힘, 화천대유 특별법안 제출",https://news.naver.com/main/read.naver?mode=LS...


In [50]:
url = df.loc[0, 'link']
response = requests.get(url, headers=headers)
dom = BeautifulSoup(response.text, "html.parser")
content = dom.select_one("#articleBodyContents").text.strip()
content = content.replace("\wa0", " ")
content[:100]

'이명박·박근혜 두 전직 대통령을 ‘사면해서는 안 된다’는 응답이 ‘사면해야 한다’는 응답보다 많다는 여론조사 결과가 나왔다.한국갤럽이 5일 발표한 11월 1주 여론조사 결과에 따르'

In [54]:
df['content'] = ""
df.head(2)

,title,link,content
0,"‘MB·박근혜 사면’ 반대 48%, 찬성 44%",https://news.naver.com/main/read.naver?mode=LS...,
1,"보훈처, 보훈문화 확산 위한 '보훈문화포럼' 개최",https://news.naver.com/main/read.naver?mode=LS...,


In [55]:
for idx, data in df.iterrows():
    response = requests.get(data.link, headers=headers)
    dom = BeautifulSoup(response.text, "html.parser")
    content = dom.select_one("#articleBodyContents").text.strip()
    content = content.replace("\wa0", " ")
    df.loc[idx, 'content'] = content

In [59]:
df.replace("\t", " ").replace("\n", " ").head(2)

,title,link,content
0,"‘MB·박근혜 사면’ 반대 48%, 찬성 44%",https://news.naver.com/main/read.naver?mode=LS...,이명박·박근혜 두 전직 대통령을 ‘사면해서는 안 된다’는 응답이 ‘사면해야 한다’는...
1,"보훈처, 보훈문화 확산 위한 '보훈문화포럼' 개최",https://news.naver.com/main/read.naver?mode=LS...,'보훈교육 활성화' '보훈문화진흥원 설립 방안' 등 논의 5일 정부세종청사 대강당에...


In [60]:
df.to_csv("article.csv", index=False)

In [ ]:
read.csv(article.csv)